In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, optimizers
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
# Definir las rutas de los directorios de entrenamiento y prueba
train_dir = 'Cars Dataset Augmented/train'
test_dir = 'Cars Dataset Augmented/test'
val_dir = 'Cars Dataset Augmented/val'

IMG_SIZE=128
# Define the target number of images for each split
target_train_images = 820
target_test_images = 200

# Data augmentation parameters
datagen = ImageDataGenerator(
							rescale = 1./255,
							rotation_range = 10,
							width_shift_range = 0.1,
							height_shift_range = 0.1,
						  shear_range = 0.25,
							zoom_range = 0.3,
							horizontal_flip = True)

def augment_images_to_target(directory, target_count):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
            current_count = len(images)
            print(f"Class: {class_name}, Current images: {current_count}, Target images: {target_count}")

            if current_count < target_count:
                needed_count = target_count - current_count
                print(f"Generating {needed_count} augmented images for {class_name}")

                # Determine how many augmented images to generate per original image
                # We want to generate needed_count in total, so distribute across existing images
                if current_count > 0:
                    aug_per_image = max(1, needed_count // current_count)
                else:
                    # If there are no original images, augmentation is not possible
                    print(f"No original images in {class_name} to augment.")
                    continue

                generated_count = 0
                for img_path in images:
                    if generated_count >= needed_count:
                        break # Stop if we have generated enough images

                    img = load_img(img_path)
                    x = img_to_array(img)
                    x = x.reshape((1,) + x.shape)  # Reshape to (1, height, width, channels)

                    # Generate augmented images
                    i = 0
                    for batch in datagen.flow(x, batch_size=1,
                                              save_to_dir=class_dir,
                                              save_prefix=f'dt_aug_{class_name}_',
                                              save_format='jpeg'):
                        i += 1
                        generated_count += 1
                        if i >= aug_per_image or generated_count >= needed_count:
                            break  # Generate a certain number of augmented images per original image

                print(f"Finished augmenting for {class_name}. Total images after augmentation: {len(os.listdir(class_dir))}")

# Augment the training data
print("Augmenting training data...")
augment_images_to_target(train_dir, target_train_images)

# Augment the test data
print("\nAugmenting test data...")
augment_images_to_target(test_dir, target_test_images)

print("\nData augmentation complete.")

Augmenting training data...
Class: Audi, Current images: 820, Target images: 820
Class: Hyundai Creta, Current images: 810, Target images: 820
Generating 10 augmented images for Hyundai Creta
Finished augmenting for Hyundai Creta. Total images after augmentation: 818
Class: Mahindra Scorpio, Current images: 814, Target images: 820
Generating 6 augmented images for Mahindra Scorpio
Finished augmenting for Mahindra Scorpio. Total images after augmentation: 820
Class: Rolls Royce, Current images: 808, Target images: 820
Generating 12 augmented images for Rolls Royce
Finished augmenting for Rolls Royce. Total images after augmentation: 820
Class: Swift, Current images: 816, Target images: 820
Generating 4 augmented images for Swift
Finished augmenting for Swift. Total images after augmentation: 820
Class: Tata Safari, Current images: 819, Target images: 820
Generating 1 augmented images for Tata Safari
Finished augmenting for Tata Safari. Total images after augmentation: 820
Class: Toyota 

# Contar Imagenes en cada carpeta

In [5]:
from prettytable import PrettyTable

def count_files_in_folders(directory):
  """
  Counts the number of files in each subfolder of a given directory.

  Args:
    directory (str): The path to the directory to analyze.

  Returns:
    list: A list of dictionaries, where each dictionary contains the
          'Folder Name' and 'File Count' for a subfolder.
  """
  folder_data = []
  for root, dirs, files in os.walk(directory):
    file_count = len(files)
    folder_data.append({'Folder Name': os.path.relpath(root, directory), 'File Count': file_count})
  return folder_data

def print_file_count_table(directory):
  """
  Prints a table showing the file count for each subfolder.

  Args:
    directory (str): The path to the directory to analyze.
  """
  folder_data = count_files_in_folders(directory)

  table = PrettyTable()
  table.field_names = ["Folder Name", "File Count"]

  for data in folder_data:
    table.add_row([data['Folder Name'], data['File Count']])

  print(table)

# Example usage with your train and test directories:
print("File counts in train directory:")
print_file_count_table(train_dir)

print("\nFile counts in test directory:")
print_file_count_table(test_dir)

print("\nFile counts in val directory:")
print_file_count_table(val_dir)


File counts in train directory:
+------------------+------------+
|   Folder Name    | File Count |
+------------------+------------+
|        .         |     0      |
|       Audi       |    820     |
|  Hyundai Creta   |    818     |
| Mahindra Scorpio |    820     |
|   Rolls Royce    |    820     |
|      Swift       |    820     |
|   Tata Safari    |    820     |
|  Toyota Innova   |    820     |
+------------------+------------+

File counts in test directory:
+------------------+------------+
|   Folder Name    | File Count |
+------------------+------------+
|        .         |     0      |
|       Audi       |    200     |
|  Hyundai Creta   |    200     |
| Mahindra Scorpio |    200     |
|   Rolls Royce    |    200     |
|      Swift       |    200     |
|   Tata Safari    |    200     |
|  Toyota Innova   |    200     |
+------------------+------------+

File counts in val directory:
+------------------+------------+
|   Folder Name    | File Count |
+------------------+-